In [3]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
# Importing the dataset for working and gaining its information
credit_df = pd.read_csv("credit_fraud_data.csv")
credit_df.info()
credit_df.shape
credit_df['Class'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

Class
0    284315
1       492
Name: count, dtype: int64

In [5]:
# Grouping Legit and Fraud transactions 
truetransac = credit_df[credit_df.Class == 0]
falsetransac = credit_df[credit_df.Class == 1]
print(truetransac.shape, falsetransac.shape)
truetransac.Amount.describe()
falsetransac.Amount.describe()

(284315, 31) (492, 31)


count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [6]:
credit_df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [7]:
# UnderSampling of both transactions and concatenating to make a new dataset
truetransac_sample = truetransac.sample(n=492)
new_df = pd.concat([truetransac_sample,falsetransac], axis =0)
new_df['Class'].value_counts()


Class
0    492
1    492
Name: count, dtype: int64

In [8]:
new_df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,92289.117886,-0.051294,0.016277,0.126450,-0.007695,0.038586,0.044726,0.046171,0.026093,0.089729,...,0.013811,0.019289,-0.025437,0.047218,0.035718,-0.030929,-0.017315,-0.012586,0.023899,84.779431
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [9]:
# Grouping the Legit and Fraud transactions using the new data
A = new_df.drop(columns = 'Class', axis = 1)
C = new_df['Class']
A_train, A_test, C_train, C_test = train_test_split(A, C, test_size=0.3, stratify=C, random_state = 5)
print(A.shape,A_train.shape,A_test.shape)

(984, 30) (688, 30) (296, 30)


In [10]:
# Fitting and Training the data into the Model 
model = LogisticRegression(max_iter=200)
model.fit(A_train,C_train)
A_train_predict = model.predict(A_train)
training_accuracy = accuracy_score(A_train_predict, C_train)
# Accuracy of the train data
print("Accuracy of training data: ", training_accuracy) 

Accuracy of training data:  0.9505813953488372


/home/harshil/miniconda3/envs/torch/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
# Accuracy of the test data
model.fit(A_test, C_test)
A_test_predict = model.predict(A_test)
test_accuracy = accuracy_score(A_test_predict, C_test)
print("Accuracy of test data: ", test_accuracy)

Accuracy of test data:  0.9695945945945946


/home/harshil/miniconda3/envs/torch/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
